# Q1

H0 : False H1 : True<br>
실제 false / 예측 : True -> FP<br>
실제 True / 예측 : false -> FN<br>

Type 1 / Type 2 는 trade off 관계. 즉 FP.FN 도 Trade off.

# Q2-1

Accuracy(정확도: 전체 예측 중 옳게 말한 결과의 비율) = (TP+TN) / (TP+TN+FP+FN)

Precision(정밀도: True라고 예측한 것 중 실제로 True의 비율) = TP / (TP+FP)

Recall(재현율: 실제 True 중 True라고 예측한 것의 비율) = TP / (TP+FN)

# Q2-2

도둑탐지모델 -> 도둑(T), 시민(F) 에 대해서 실제 도둑인사람 중에서 모델이 도둑이라고 예측한 비율인 재현율이 매우 중요할것이다. (도둑은 다 잡아야하니까)

threshold 를 낮추면, 도둑(T) 에 대한 조건이 낮아지므로 많은 도둑을 잡을 수 있을것이다. (재현율은 높음)

몰론 어느정도의 시민들이 억울하게 도둑으로 분류되긴 하겠지만, 어쩃든 도둑은 모두 잡게될거니까~

# Q3 주석달기(sgd)

In [38]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 


In [39]:
# MNIST Dataset (Images and Labels)  # 데이터셋 불러오기
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 

test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, # 데이터로더 만들기. 
                                           batch_size = batch_size,  
                                           shuffle = True) # shuffle 을 True 로 주어서 학습떄마다 데이터셋이 섞이게 한다.
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)  # test 시에는 섞이지 않음


In [40]:
# Hyper Parameters  
input_size = 784 # input x 데이터의 특성이 784개 이므로 784
num_classes = 10 # y 의 클래스가 10개
num_epochs = 5
batch_size = 100
learning_rate = 0.001


In [42]:
class LogisticRegression(nn.Module):  # calss 를 이용해 model 을 정의하자.
    def __init__(self, input_size, num_classes):  #model 의 forward 계산에 쓰이게 되는 model 들을 정의한다.
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) # nn 에서 직접 선형모델 호출
    def forward(self, x):  #forward 계산 메서드 정의
        out = self.linear(x)  #층이 한개이므로 그냥 모델을썻따
        return out 


In [43]:
model = LogisticRegression(input_size, num_classes) # 모델선언

In [44]:
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 

In [47]:
# Training the Model 
for epoch in range(num_epochs): # epoch 만큼 반복
    for i, (images, labels) in enumerate(train_loader): # enumerate 를 통해서 batch 동안의 모델향상도 보자! (데이터 크기가 크면 enumerate 는 쓰지않는듯)
        images = Variable(images.view(-1, 28 * 28))  # 데이터 전처리작업때 , numpy 를 받았기떄문에, pytorch 에서 쓸 수 있게 tensor 로 변환
        labels = Variable(labels) 
        
       # Forward + Backward + Optimize 
        optimizer.zero_grad() # 기울기초기화
        outputs = model(images) # 예측치
        loss = criterion(outputs, labels) # loss 선언
        loss.backward() # 기울기계산
        optimizer.step() #loss 선언
        
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, len(train_dataset) // batch_size, loss.data.item())) 



Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.0645
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 1.9261
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.8975
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.7934
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.7435
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.6853
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.5432
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.5305
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.4638
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5137
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.4486
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4587
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.2870
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.3247
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.1894
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2553
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.2380
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2500
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2181
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.0527
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [51]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) # test 에서도 똑같은 작업 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) # ㅡmax 의 label 로 predict
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
    
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 


Accuracy of the model on the 10000 test images:  83 %


# rmsprop

In [53]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 
  
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data', 
                            train = True, 
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data', 
                           train = False, 
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                           batch_size = batch_size, 
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
                                          batch_size = batch_size, 
                                          shuffle = False) 
  
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001
  
# Model 
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
        
    def forward(self, x): 
        out = self.linear(x) 
        return out 
    
    
model = LogisticRegression(input_size, num_classes) 
  
# Loss and Optimizer 
# Softmax is internally computed. 
# Set parameters to be updated. 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate) 
  
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
        
        # Forward + Backward + Optimize 
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
        
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data.item())) 
            
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
    
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 


Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 0.5665
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 0.4053
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 0.3134
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 0.2726
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 0.2694
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 0.3428
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 0.2242
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 0.2783
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 0.3588
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 0.4319
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 0.3311
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 0.2018
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 0.3667
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 0.3379
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 0.3824
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 0.2053
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 0.3399
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 0.3295
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 0.2630
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 0.2259
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

# adam

In [54]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 
  
# MNIST Dataset (Images and Labels) 
train_dataset = dsets.MNIST(root ='./data', 
                            train = True, 
                            transform = transforms.ToTensor(), 
                            download = True) 
  
test_dataset = dsets.MNIST(root ='./data', 
                           train = False, 
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                           batch_size = batch_size, 
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, 
                                          batch_size = batch_size, 
                                          shuffle = False) 
  
# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001
  
# Model 
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
        
    def forward(self, x): 
        out = self.linear(x) 
        return out 
    
    
model = LogisticRegression(input_size, num_classes) 
  
# Loss and Optimizer 
# Softmax is internally computed. 
# Set parameters to be updated. 
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 
  
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) 
        labels = Variable(labels) 
        
        # Forward + Backward + Optimize 
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
        
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data.item())) 
            
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
    
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 


Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 0.8766
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 0.6384
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 0.5750
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 0.4537
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 0.4598
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 0.3198
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 0.4135
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 0.3853
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 0.3956
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 0.3342
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 0.3423
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 0.2762
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 0.4759
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 0.3590
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 0.4757
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 0.1145
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 0.3246
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 0.4850
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 0.3746
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 0.3525
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

# SGD 바꾸기

cost = F.mse_loss(prediction, y_train)

cost 가 mse이다. 즉 gradient 를 구하기가 매우 쉬울것이다.

In [56]:
# 데이터 
x_train = torch.FloatTensor([[1], [2], [3]]) 
y_train = torch.FloatTensor([[1], [2], [3]])  

In [58]:
# For reproducibility 
torch.manual_seed(1)

# 모델 초기화 
W=torch.zeros(1)
b=torch.zeros(1)

# optimizer 설정 
lr=0.01

nb_epochs=1000
for epoch in range(nb_epochs + 1):        

    # H(x) 계산    
    prediction = (x_train * W) + b       

    # cost 계산    
    cost = F.mse_loss(prediction, y_train)  # 이 mse loss 를 미분한다는것은 (wx+b -y)^2 미분이다.(by w,b)       
    w_gradient=torch.sum(((W * x_train + b)- y_train) * x_train)
    b_gradient=torch.sum((W*x_train + b) - y_train)
    
    #update w , b
    W-=lr*w_gradient
    b-=lr*b_gradient
    
    # 100번마다 로그 출력                 
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()        
        ))

Epoch    0/1000 W: 0.140, b: 0.060 Cost: 4.666667
Epoch  100/1000 W: 0.887, b: 0.256 Cost: 0.009462
Epoch  200/1000 W: 0.922, b: 0.178 Cost: 0.004594
Epoch  300/1000 W: 0.945, b: 0.124 Cost: 0.002231
Epoch  400/1000 W: 0.962, b: 0.087 Cost: 0.001083
Epoch  500/1000 W: 0.973, b: 0.060 Cost: 0.000526
Epoch  600/1000 W: 0.982, b: 0.042 Cost: 0.000255
Epoch  700/1000 W: 0.987, b: 0.029 Cost: 0.000124
Epoch  800/1000 W: 0.991, b: 0.020 Cost: 0.000060
Epoch  900/1000 W: 0.994, b: 0.014 Cost: 0.000029
Epoch 1000/1000 W: 0.996, b: 0.010 Cost: 0.000014
